In [1]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division

import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
import string
import pandas as pd
from scipy import stats

%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs

import json
import pymongo as pm

from svgpathtools import parse_path

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

## purpose

#### upload sketches to S3 [maybe do this later]
#### build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [2]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_allcats.csv'
_meta = pd.read_csv(path_to_metadata)   ### raw meta with all categories


In [3]:
# print first few rows of meta
_meta.head()

,gameID,trialNum,condition,target,drawDuration,outcome,response,numStrokes,Distractor1,Distractor2,Distractor3,pose,iteration,svg,viewerRT,mean_intensity,category
0,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,1,further,knob,18.389,True,knob,7,crow,redsport,basset,35,pilot2,"[u'M69,167c0,27.13435 5,54.84972 5,81c0,1.8150...",6.656,0.0347,chair
1,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,2,further,redsport,2.820,True,redsport,3,crow,knob,basset,35,pilot2,"[u'M51,224c0,-65.25556 75.24654,-35.24654 83,-...",5.607,0.0320,car
2,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,3,closer,bluesport,22.093,False,beetle,5,beetle,hatchback,bluesedan,35,pilot2,"[u'M34,212c5.3353,-10.6706 6.56215,-22.81075 9...",6.695,0.0466,car
3,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,4,closer,chihuahua,24.122,True,chihuahua,14,weimaraner,doberman,pug,35,pilot2,"[u'M99,157c-26.4479,17.63193 -2.9121,54.60807 ...",3.133,0.0371,dog
4,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,5,closer,pug,34.260,False,chihuahua,12,weimaraner,doberman,chihuahua,35,pilot2,"[u'M69,172c0,4.72338 -5.37528,22.62472 -2,26c9...",5.739,0.0381,dog


In [4]:
## TODO: what we actually want Make sure that the data we're excluding from annotation 
## really the come from the trials where we had the shift-key artifact. 
## Right now, we are excluding sketches with numStrokes > mu + 3*sd, which is an imperfect proxy for that.
mu = np.mean(_meta['numStrokes'])
sd = np.std(_meta['numStrokes'])
thresh = mu + 3*sd
_meta = _meta[_meta['numStrokes']<thresh]
_meta.reset_index(inplace=True) ## reset index on meta_chairs

## subset by chairs
meta_chairs = _meta[_meta['category']=='chair'] ### subsetted meta with just chairs

## sub-select good 4 chairs
## inlay>waiting>straight>leather
chairs4_list = ['inlay','waiting','straight','leather']
meta_chairs4 = meta_chairs[meta_chairs['target'].isin(chairs4_list)]
meta_chairs4.reset_index(inplace=True)

## assign which meta we will actually upload to mongo in this session
category_flag = 'allcats' ## options: ['allcats','chairs','chairs4']

if category_flag == 'chairs':
    meta = meta_chairs
elif category_flag == 'chairs4':
    meta = meta_chairs4
elif category_flag == 'allcats':
    meta = _meta

In [5]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append(["backrest,armrest,seat,leg"])        
    if meta.category[i]=="dog":
        parts.append(["eye,mouth,ear,head,neck,body,leg,paw,tail"])
    if meta.category[i] == "bird":
        parts.append(["eye,beak,head,body,wing,leg,feet,tail"])
    if meta.category[i] == "car":
        parts.append(["bumper,headlight,hood,windshield,window,body,door,trunk,wheel"])
meta = meta.assign(parts=pd.Series(parts))

In [6]:
## add iteration name information
_iterationName = 'sketchpad_basic_{}'.format(category_flag)
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [7]:
print meta.columns

Index([u'index', u'gameID', u'trialNum', u'condition', u'target',
       u'drawDuration', u'outcome', u'response', u'numStrokes', u'Distractor1',
       u'Distractor2', u'Distractor3', u'pose', u'iteration', u'svg',
       u'viewerRT', u'mean_intensity', u'category', u'parts',
       u'iterationName'],
      dtype='object')


In [8]:
## svg string formatting
svg = []
for i,d in meta.iterrows():    
    splitted = d['svg'].split("'") ## parse string to re-split up into strokes
    svgString = [i for i in splitted if i[0]=='M'] ## check to make sure it is a real start of a spline
    svg.append(svgString)
meta = meta.assign(svg=pd.Series(svg)) 

In [9]:
## add numSplines to the meta data
numSplines = []
for sk_ind, sketch in enumerate(meta.svg.values):
    num_splines = 0
    for stroke_ind,stroke in enumerate(sketch):
        parsed = parse_path(stroke)
        num_splines += len(parsed)
    numSplines.append(num_splines)
meta = meta.assign(numSplines=pd.Series(numSplines))     

In [10]:
## add empty games list
games = [[] for i in np.arange(len(meta))]
meta = meta.assign(games=pd.Series(games))

In [25]:
## filter so we can cover all the sketches multiple times with a budget of around $500 
budget_cap = 500
base_pay = 1.00
expected_bonus = 0.25
amt_commission = 1.2
mean_cost_single_session = (base_pay + expected_bonus) * amt_commission
print 'We expect to pay out approx. ${:.2f} per session, including commission.'.format(mean_cost_single_session)
num_sessions = int(np.floor(budget_cap/mean_cost_single_session))
print 'We have enough money to run approx. {} sessions.'.format(num_sessions)
num_total_annotation_trials_in_budget = num_sessions * 10
num_times_per_sketch = 3
num_unique_sketches_annotatable = num_total_annotation_trials_in_budget/3
print 'We can afford to annotation approx. {} sketches {} times.'.format(int(num_unique_sketches_annotatable), num_times_per_sketch)

We expect to pay out approx. $1.50 per session, including commission.
We have enough money to run approx. 333 sessions.
We can afford to annotation approx. 1110 sketches 3 times.


In [84]:
### maybe filter on games that had both high accuracy and big context effect (big diff in num strokes between close & far)

## get list of games sorted by accuracy
games_sorted_by_accuracy = meta.groupby(['gameID'])['outcome'].mean().reset_index().sort_values('outcome',ascending=False).gameID.values

## get list of games sorted by how many more strokes were used in close condition than far condition
# reshape dataframe that computes mean strokes for each condition for each game
mean_strokes_in_condition_per_game = meta.groupby(['gameID','condition'])['numStrokes'].mean().reset_index().sort_values('gameID',ascending=True)
mscg = mean_strokes_in_condition_per_game
# pivots table to make two columns for each condition
msg = mscg.pivot(index='gameID', columns='condition')['numStrokes'].reset_index()
# assigns new column with close vs. far diff, then sorts
msg['diff'] = msg.apply(lambda x: x['closer'] - x['further'],axis=1)
msg = msg.sort_values('diff',ascending=False).reset_index()
# get list of games sorted by difference in num strokes used in each condition
games_sorted_by_contextDiff = msg['gameID'].values

In [ ]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta_{}.js'.format(category_flag), 'w') as fout:
     json.dump(stimdict, fout)

### interlude to examine detailed statistics on constituent splines

In [ ]:
## svg is a list of sketches
## each entry contains a list of strokes
## first let's convert into absolute coordinates
## then let's convert these into a list of splines that are "long enough"

In [ ]:
def convert_relative_spline_to_absolute(parsed):
    svg_abs = ''
    for i,p in enumerate(parsed):
        if len(p)==4: ## cubic bezier
            svg_abs += ' M '
            svg_abs += '{},{}'.format(str(p.start.real),str(p.start.imag))
            svg_abs += ' C'
            svg_abs += ' {},{}'.format(str(p.control1.real),str(p.control1.imag))
            svg_abs += ' {},{}'.format(str(p.control2.real),str(p.control2.imag))
            svg_abs += ' {},{}'.format(str(p.end.real),str(p.end.imag)) 
        if len(p)==2: ## line segment
            svg_abs += ' M '
            svg_abs += '{},{}'.format(str(p.start.real),str(p.start.imag))
            svg_abs += ' L'
            svg_abs += ' {},{}'.format(str(p.end.real),str(p.end.imag))          
#     assert np.all(np.round(parsed)==np.round(parse_path(svg_abs)))==True
    return svg_abs

In [ ]:
## get list of sketch svg converted to absolute coordinates
## grouped into **strokes**
svg_abs_strokes = []
for this_sketch in svg:
    sketch_abs = []
    for this_stroke in this_sketch:  
        this_stroke = this_stroke.replace('v0','') ## eliminate single points
        this_stroke = this_stroke.replace('h0','') ## eliminate single points
        parsed = parse_path(this_stroke)
        parsed_abs = convert_relative_spline_to_absolute(parsed)
        sketch_abs.append(parsed_abs)
    svg_abs_strokes.append(sketch_abs)

In [ ]:
## get list of sketch svg converted to absolute coordinates
## grouped into **splines**
svg_abs_splines = [] 
stroke_num_within_sketch = [] ## get stroke num within sketch
for this_sketch in svg_abs_strokes:
    sketch_abs = []
    _stroke_num_within_sketch = []    
    for stroke_id,this_stroke in enumerate(this_sketch):
        this_path = parse_path(this_stroke)
        for i,p in enumerate(this_path):
            _svg_abs = ''
            if len(p)==4: ## cubic bezier
                _svg_abs += ' M '
                _svg_abs += '{},{}'.format(str(p.start.real),str(p.start.imag))
                _svg_abs += ' C'
                _svg_abs += ' {},{}'.format(str(p.control1.real),str(p.control1.imag))
                _svg_abs += ' {},{}'.format(str(p.control2.real),str(p.control2.imag))
                _svg_abs += ' {},{}'.format(str(p.end.real),str(p.end.imag)) 
            if len(p)==2: ## line segment
                _svg_abs += ' M '
                _svg_abs += '{},{}'.format(str(p.start.real),str(p.start.imag))
                _svg_abs += ' L'
                _svg_abs += ' {},{}'.format(str(p.end.real),str(p.end.imag))  
            sketch_abs.append(_svg_abs)
            _stroke_num_within_sketch.append(stroke_id)
    svg_abs_splines.append(sketch_abs)
    stroke_num_within_sketch.append(_stroke_num_within_sketch)

In [ ]:
## create list of spline arc lengths nested in the same way as svg_abs_splines
svg_abs_spline_lengths = []
for sketch_ind,this_sketch in enumerate(svg_abs_splines):
    sketch_abs_length = []
    for spline_ind,spline in enumerate(this_sketch):
        curr_stroke_ind = stroke_num_within_sketch[sketch_ind][spline_ind]    
        curr_spline_length = parse_path(spline).length()
        sketch_abs_length.append(curr_spline_length)
    svg_abs_spline_lengths.append(sketch_abs_length)

In [ ]:
def flatten(x):
    return [item for sublist in x for item in sublist]

In [ ]:
## get all spline lengths
flat_spline_lengths = flatten(svg_abs_spline_lengths)

## make figure
plt.figure(figsize=(12,4))
sns.set_context('paper')
plt.title('Distribution of spline arc lengths across all sketches')
plt.hist(flat_spline_lengths,200)
plt.xticks(np.linspace(0,200,51))
plt.xlim(0,200)

### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [ ]:
## do you want to upload a "development mode" collection for testing?
dev_mode = True 

In [ ]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta_{}.js'.format(category_flag),mode='ru').read())
assert len(J)==len(meta)

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [ ]:
## define the dbname and collection name
db = conn['stimuli']
if dev_mode:
    coll = db['svg_annotation_sketchpad_basic_{}_dev'.format(category_flag)]
else:
    coll = db['svg_annotation_sketchpad_basic_{}'.format(category_flag)]

In [ ]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)
reallyRun = False        

In [ ]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))

### check how many sketches have been tagged in the collection how many times

In [ ]:
cumulative = []
print 'There are: '
for i in np.arange(10):
    print '{} sketches that have been retrieved {} times from the database.'.format(coll.find({'games': {'$size': i}}).count(),i)
    cumulative.append(coll.find({'games': {'$size': i}}).count())
print 'While not guaranteed, sketches retrieved from the database are usually annotated, so this is a reasonable proxy for how many sketches have been annotated.'